In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


/var/folders/fw/slnm06yn15lgwlcg_7jpt3t00000gn/T/ipykernel_90085/2257586243.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


In [3]:
import pandas as pd

df = pd.read_sql_query("SELECT * \
                FROM Facilities;"
                      ,engine)
print(df)


   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   
5      5   Massage Room 2         9.9       80.0           4000   
6      6     Squash Court         3.5       17.5           5000   
7      7    Snooker Table         0.0        5.0            450   
8      8       Pool Table         0.0        5.0            400   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  
5                3000  
6                  80  
7                  15  
8                  15  


In [4]:
# Q1: Some of the facilities charge a fee to members, but some do not.
# Write a SQL query to produce a list of the names of the facilities that do.


df = pd.read_sql_query("SELECT name \
                FROM Facilities \
                WHERE membercost > 0;"
                      ,engine)
print(df)


             name
0  Tennis Court 1
1  Tennis Court 2
2  Massage Room 1
3  Massage Room 2
4    Squash Court


In [5]:
# Q2: How many facilities do not charge a fee to members?

df = pd.read_sql_query("SELECT name \
                FROM Facilities \
                WHERE membercost = 0;"
                      ,engine)

print(df)

              name
0  Badminton Court
1     Table Tennis
2    Snooker Table
3       Pool Table


In [6]:
# Q3: Write an SQL query to show a list of facilities that charge a fee to members,
# where the fee is less than 20% of the facility's monthly maintenance cost.
# Return the facid, facility name, member cost, and monthly maintenance of the
# facilities in question.

df = pd.read_sql_query("SELECT facid, name, membercost, monthlymaintenance \
                FROM Facilities \
                WHERE membercost > 0 \
                AND membercost < monthlymaintenance*0.2;"
                      ,engine)

print(df)

   facid            name  membercost  monthlymaintenance
0      0  Tennis Court 1         5.0                 200
1      1  Tennis Court 2         5.0                 200
2      4  Massage Room 1         9.9                3000
3      5  Massage Room 2         9.9                3000
4      6    Squash Court         3.5                  80


In [7]:
# Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
# Try writing the query without using the OR operator.

df = pd.read_sql_query("SELECT * \
                FROM Facilities \
                WHERE facid IN (1, 5);"
                      ,engine)

print(df)

   facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0         25           8000   
1      5  Massage Room 2         9.9         80           4000   

   monthlymaintenance  
0                 200  
1                3000  


In [8]:
# Q5: Produce a list of facilities, with each labelled as
# 'cheap' or 'expensive', depending on if their monthly maintenance cost is
# more than $100. Return the name and monthly maintenance of the facilities
# in question.

df = pd.read_sql_query("SELECT name, monthlymaintenance,  \
                        CASE WHEN monthlymaintenance > 100 THEN 'expensive' \
                             ELSE 'cheap' END \
                             AS maintenance_level \
                        FROM Facilities"
                        ,engine)



print(df)

              name  monthlymaintenance maintenance_level
0   Tennis Court 1                 200         expensive
1   Tennis Court 2                 200         expensive
2  Badminton Court                  50             cheap
3     Table Tennis                  10             cheap
4   Massage Room 1                3000         expensive
5   Massage Room 2                3000         expensive
6     Squash Court                  80             cheap
7    Snooker Table                  15             cheap
8       Pool Table                  15             cheap


In [9]:
# Q6: You'd like to get the first and last name of the last member(s)
# who signed up. Try not to use the LIMIT clause for your solution.

df = pd.read_sql_query('SELECT firstname, surname \
                        FROM Members \
                        WHERE joindate = \
                           (SELECT MAX(joindate) \
                           FROM Members);'\
                       ,engine)
print(df)

  firstname surname
0    Darren   Smith


In [10]:
# Q7: Produce a list of all members who have used a tennis court.
# Include in your output the name of the court, and the name of the member
# formatted as a single column. Ensure no duplicate data, and order by
# the member name.

df = pd.read_sql_query('SELECT DISTINCT m.firstname || m.surname AS member_name, \
                            f.name AS court_name \
                        FROM Bookings AS b \
                        INNER JOIN Facilities AS f \
                            ON b.facid = f.facid \
                        INNER JOIN Members AS m \
                            ON b.memid = m.memid \
                        WHERE f.name LIKE "Tennis Court%" \
                        AND m.memid <> 0 \
                        ORDER BY member_name;'\
                       ,engine)

print(df)

         member_name      court_name
0          AnneBaker  Tennis Court 1
1          AnneBaker  Tennis Court 2
2        BurtonTracy  Tennis Court 2
3        BurtonTracy  Tennis Court 1
4        CharlesOwen  Tennis Court 1
5        CharlesOwen  Tennis Court 2
6        DarrenSmith  Tennis Court 2
7       DavidFarrell  Tennis Court 1
8       DavidFarrell  Tennis Court 2
9         DavidJones  Tennis Court 2
10        DavidJones  Tennis Court 1
11       DavidPinker  Tennis Court 1
12      DouglasJones  Tennis Court 1
13      EricaCrumpet  Tennis Court 1
14     FlorenceBader  Tennis Court 2
15     FlorenceBader  Tennis Court 1
16     GeraldButters  Tennis Court 1
17     GeraldButters  Tennis Court 2
18   HenriettaRumney  Tennis Court 2
19         JackSmith  Tennis Court 1
20         JackSmith  Tennis Court 2
21    JaniceJoplette  Tennis Court 1
22    JaniceJoplette  Tennis Court 2
23     JemimaFarrell  Tennis Court 2
24     JemimaFarrell  Tennis Court 1
25        JoanCoplin  Tennis Court 1
2

In [11]:
# Q8: Produce a list of bookings on the day of 2012-09-14 which
# will cost the member (or guest) more than $30. Remember that guests have
# different costs to members (the listed costs are per half-hour 'slot'), and
# the guest user's ID is always 0. Include in your output the name of the
# facility, the name of the member formatted as a single column, and the cost.
# Order by descending cost, and do not use any subqueries.

df = pd.read_sql_query('SELECT DISTINCT m.firstname || m.surname AS member_name, \
                            f.name AS court_name, \
                            CASE WHEN m.memid = 0 THEN b.slots * f.guestcost \
                                 ELSE b.slots * f.membercost END \
                                 AS cost \
                        FROM Bookings AS b \
                        INNER JOIN Facilities AS f \
                            ON b.facid = f.facid \
                        INNER JOIN Members AS m \
                            ON b.memid = m.memid \
                        WHERE b.starttime LIKE "2012-09-14%" \
                        AND ((m.memid = 0 AND (b.slots * f.guestcost) < 30) \
                        OR (m.memid > 0 AND (b.slots * f.membercost) < 30)) \
                        ORDER BY cost DESC;'\
                       ,engine)

#df = pd.read_sql_query('SELECT * \
#                        FROM Bookings AS b \
#                        INNER JOIN Facilities AS f \
#                            ON b.facid = f.facid \
#                        INNER JOIN Members AS m \
#                            ON b.memid = m.memid;'\
#                       ,engine)

print(df)

        member_name       court_name  cost
0         JackSmith   Massage Room 1  19.8
1     JemimaFarrell   Massage Room 1  19.8
2    PonderStibbons   Massage Room 1  19.8
3       BurtonTracy   Massage Room 1  19.8
4    MatthewGenting   Massage Room 1  19.8
5     FlorenceBader   Massage Room 2  19.8
6       BurtonTracy   Tennis Court 1  15.0
7       DavidPinker   Tennis Court 1  15.0
8     GeraldButters   Tennis Court 1  15.0
9         TimRownam   Tennis Court 1  15.0
10     DouglasJones   Tennis Court 1  15.0
11        AnneBaker     Squash Court   7.0
12     TimothyBaker     Squash Court   7.0
13      DavidPinker     Squash Court   7.0
14      DarrenSmith  Badminton Court   0.0
15    AnnaMackenzie  Badminton Court   0.0
16    GeraldButters  Badminton Court   0.0
17   PonderStibbons  Badminton Court   0.0
18    FlorenceBader     Table Tennis   0.0
19     TimothyBaker     Table Tennis   0.0
20   MatthewGenting     Table Tennis   0.0
21    AnnaMackenzie     Table Tennis   0.0
22      Cha

In [12]:
# Q9: This time, produce the same result as in Q8, but using a subquery.

df = pd.read_sql_query('SELECT DISTINCT m.firstname || m.surname AS member_name, \
                            f.name AS court_name, \
                            CASE WHEN m.memid = 0 THEN b.slots * f.guestcost \
                                 ELSE b.slots * f.membercost END \
                                 AS cost \
                        FROM (SELECT * \
                              FROM Bookings \
                              WHERE starttime LIKE "2012-09-14%") \
                              AS b \
                        INNER JOIN Facilities AS f \
                            ON b.facid = f.facid \
                        INNER JOIN Members AS m \
                            ON b.memid = m.memid \
                        WHERE ((m.memid = 0 AND (b.slots * f.guestcost) < 30) \
                        OR (m.memid > 0 AND (b.slots * f.membercost) < 30)) \
                        ORDER BY cost DESC;'\
                       ,engine)


print(df)

        member_name       court_name  cost
0         JackSmith   Massage Room 1  19.8
1     JemimaFarrell   Massage Room 1  19.8
2    PonderStibbons   Massage Room 1  19.8
3       BurtonTracy   Massage Room 1  19.8
4    MatthewGenting   Massage Room 1  19.8
5     FlorenceBader   Massage Room 2  19.8
6       BurtonTracy   Tennis Court 1  15.0
7       DavidPinker   Tennis Court 1  15.0
8     GeraldButters   Tennis Court 1  15.0
9         TimRownam   Tennis Court 1  15.0
10     DouglasJones   Tennis Court 1  15.0
11        AnneBaker     Squash Court   7.0
12     TimothyBaker     Squash Court   7.0
13      DavidPinker     Squash Court   7.0
14      DarrenSmith  Badminton Court   0.0
15    AnnaMackenzie  Badminton Court   0.0
16    GeraldButters  Badminton Court   0.0
17   PonderStibbons  Badminton Court   0.0
18    FlorenceBader     Table Tennis   0.0
19     TimothyBaker     Table Tennis   0.0
20   MatthewGenting     Table Tennis   0.0
21    AnnaMackenzie     Table Tennis   0.0
22      Cha

In [13]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members!

df = pd.read_sql_query('SELECT f.name AS facility_name, \
                            SUM(CASE WHEN m.memid = 0 THEN b.slots * f.guestcost \
                                     ELSE b.slots * f.membercost END) \
                                     AS revenue \
                        FROM Bookings AS b \
                        INNER JOIN Facilities AS f \
                            ON b.facid = f.facid \
                        INNER JOIN Members AS m \
                            ON b.memid = m.memid \
                        GROUP BY f.name \
                        HAVING revenue < 1000;' \
                        ,engine)




print(df)

   facility_name  revenue
0     Pool Table      270
1  Snooker Table      240
2   Table Tennis      180


In [14]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

df = pd.read_sql_query('SELECT m1.surname AS member_surname, \
                               m1.firstname AS member_lastname, \
                               m2.surname AS recommend_surname, \
                               m2.firstname AS recommend_firstname \
                        FROM Members AS m1 \
                        INNER JOIN Members AS m2 \
                           ON m1.memid = m2.recommendedby \
                        ORDER BY m2.surname, m2.firstname;' \
                       , engine)
print(df)

   member_surname member_lastname  recommend_surname recommend_firstname
0        Stibbons          Ponder              Bader            Florence
1        Stibbons          Ponder              Baker                Anne
2         Farrell          Jemima              Baker             Timothy
3          Rownam             Tim             Boothe                 Tim
4           Smith          Darren            Butters              Gerald
5           Baker         Timothy             Coplin                Joan
6           Smith           Tracy            Crumpet               Erica
7        Joplette          Janice               Dare               Nancy
8         Butters          Gerald            Genting             Matthew
9         Purview       Millicent               Hunt                John
10       Joplette          Janice              Jones               David
11          Jones           David              Jones             Douglas
12          Smith          Darren           Joplett

In [15]:
# Q12: Find the facilities with their usage by member, but not guests
df = pd.read_sql_query('SELECT f.name AS facility_name, \
                                SUM(slots) AS number_of_slots \
                        FROM Bookings AS b \
                        INNER JOIN Facilities AS f \
                            ON b.facid = f.facid \
                        WHERE memid > 0 \
                        GROUP BY f.name;' \
                        ,engine)

print(df)

     facility_name  number_of_slots
0  Badminton Court             1086
1   Massage Room 1              884
2   Massage Room 2               54
3       Pool Table              856
4    Snooker Table              860
5     Squash Court              418
6     Table Tennis              794
7   Tennis Court 1              957
8   Tennis Court 2              882


In [16]:
# Q13: Find the facilities usage by month, but not guests
df = pd.read_sql_query("SELECT f.name AS facility_name, \
                                SUM(slots) AS slots_per_month, \
                                STRFTIME('%Y-%m', starttime) AS year_month \
                        FROM Bookings AS b \
                        INNER JOIN Facilities AS f \
                            ON b.facid = f.facid \
                        WHERE memid > 0 \
                        GROUP BY f.name, year_month;" \
                        ,engine)

print(df)

      facility_name  slots_per_month year_month
0   Badminton Court              165    2012-07
1   Badminton Court              414    2012-08
2   Badminton Court              507    2012-09
3    Massage Room 1              166    2012-07
4    Massage Room 1              316    2012-08
5    Massage Room 1              402    2012-09
6    Massage Room 2                8    2012-07
7    Massage Room 2               18    2012-08
8    Massage Room 2               28    2012-09
9        Pool Table              110    2012-07
10       Pool Table              303    2012-08
11       Pool Table              443    2012-09
12    Snooker Table              140    2012-07
13    Snooker Table              316    2012-08
14    Snooker Table              404    2012-09
15     Squash Court               50    2012-07
16     Squash Court              184    2012-08
17     Squash Court              184    2012-09
18     Table Tennis               98    2012-07
19     Table Tennis              296    